# Data Integration

### Import libraries


In [1]:
import pandas as pd
import openpyxl
import difflib
import re
import numpy as np
import difflib



### Load Data

In [2]:
stats_lolalytics = pd.read_csv('datasets/champions_lolalytics.csv')
stats_lolalytics

,rank,champion_name,tier,lane,win_rate,pick_rate,ban_rate,pbi,games_played,tier_rank
0,1,Jax,S+,top,50.20,10.21,19.44,3,1737071,all
1,2,Fiora,S+,top,51.00,4.89,7.75,5,832573,all
2,3,Poppy,S,top,50.71,1.13,2.30,1,192796,all
3,4,Singed,S,top,52.80,1.53,0.48,4,260712,all
4,5,Zac,S,top,52.52,0.50,1.29,1,84558,all
...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,D,support,36.99,0.43,0.79,-3,3393,iron
3652,101,Lulu,D-,support,40.67,2.45,0.99,-9,19551,iron
3653,102,Yuumi,D-,support,38.84,7.27,3.04,-40,57948,iron
3654,103,Sivir,D-,support,23.92,0.11,0.93,-2,899,iron


Usiamo Lolalytics come base

### Data Cleaning

#### Columns rename

In [3]:
columns_to_rename ={
    'position': 'lane',
    'rank': 'position',
    'tier': 'grade',
    'tier_rank': 'ranking'
}

stats_lolalytics.rename(columns=columns_to_rename, inplace=True)


In [4]:
mapping_lane = {'middle': 'mid', 'bottom': 'adc'}
stats_lolalytics['lane'] = stats_lolalytics['lane'].replace(mapping_lane)
stats_lolalytics['lane'].unique()

array(['top', 'jungle', 'mid', 'adc', 'support'], dtype=object)

da sostituire con enrichment2

In [5]:
stats_blitz = pd.read_csv('datasets/enrichment_complete.csv')

In [6]:
stats_blitz

,champion_name,tier_rank,tier,counters_list,win_rate_list,role
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",TOP
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",TOP
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",TOP
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",TOP
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",TOP
...,...,...,...,...,...,...
3651,MonkeyKing,IRON,3,"['Kennen', 'Riven', 'Teemo', 'Malphite', 'Rene...","['17', '25', '26', '31', '31']",TOP
3652,MonkeyKing,IRON,3,"['Kindred', 'Lillia', 'Fiddlesticks', 'Brand',...","['30', '30', '33', '33', '33']",JUNGLE
3653,MonkeyKing,IRON,4,"['Orianna', 'Diana', 'Ekko', 'Naafiri', 'Brand']","['0', '0', '0', '0', '0']",MID
3654,Renata,IRON,2,"['Milio', 'Maokai', 'Twitch', 'Seraphine', ""Ve...","['26', '31', '33', '37', '38']",SUPPORT


In [7]:
columns_to_rename['role'] = 'lane'
stats_blitz.rename(columns=columns_to_rename, inplace=True)
stats_blitz

,champion_name,ranking,grade,counters_list,win_rate_list,lane
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",TOP
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",TOP
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",TOP
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",TOP
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",TOP
...,...,...,...,...,...,...
3651,MonkeyKing,IRON,3,"['Kennen', 'Riven', 'Teemo', 'Malphite', 'Rene...","['17', '25', '26', '31', '31']",TOP
3652,MonkeyKing,IRON,3,"['Kindred', 'Lillia', 'Fiddlesticks', 'Brand',...","['30', '30', '33', '33', '33']",JUNGLE
3653,MonkeyKing,IRON,4,"['Orianna', 'Diana', 'Ekko', 'Naafiri', 'Brand']","['0', '0', '0', '0', '0']",MID
3654,Renata,IRON,2,"['Milio', 'Maokai', 'Twitch', 'Seraphine', ""Ve...","['26', '31', '33', '37', '38']",SUPPORT


In [8]:
mapping_champion_blitz = {
    'Renata': 'Renata Glasc',
    'MonkeyKing': 'Wukong'
}

stats_blitz['champion_name'] = stats_blitz['champion_name'].replace(mapping_champion_blitz)
stats_blitz

,champion_name,ranking,grade,counters_list,win_rate_list,lane
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",TOP
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",TOP
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",TOP
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",TOP
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",TOP
...,...,...,...,...,...,...
3651,Wukong,IRON,3,"['Kennen', 'Riven', 'Teemo', 'Malphite', 'Rene...","['17', '25', '26', '31', '31']",TOP
3652,Wukong,IRON,3,"['Kindred', 'Lillia', 'Fiddlesticks', 'Brand',...","['30', '30', '33', '33', '33']",JUNGLE
3653,Wukong,IRON,4,"['Orianna', 'Diana', 'Ekko', 'Naafiri', 'Brand']","['0', '0', '0', '0', '0']",MID
3654,Renata Glasc,IRON,2,"['Milio', 'Maokai', 'Twitch', 'Seraphine', ""Ve...","['26', '31', '33', '37', '38']",SUPPORT


In [9]:
stats_blitz['lane'] = stats_blitz['lane'].str.lower()
stats_blitz

,champion_name,ranking,grade,counters_list,win_rate_list,lane
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",top
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",top
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",top
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",top
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",top
...,...,...,...,...,...,...
3651,Wukong,IRON,3,"['Kennen', 'Riven', 'Teemo', 'Malphite', 'Rene...","['17', '25', '26', '31', '31']",top
3652,Wukong,IRON,3,"['Kindred', 'Lillia', 'Fiddlesticks', 'Brand',...","['30', '30', '33', '33', '33']",jungle
3653,Wukong,IRON,4,"['Orianna', 'Diana', 'Ekko', 'Naafiri', 'Brand']","['0', '0', '0', '0', '0']",mid
3654,Renata Glasc,IRON,2,"['Milio', 'Maokai', 'Twitch', 'Seraphine', ""Ve...","['26', '31', '33', '37', '38']",support


In [10]:
stats_blitz['ranking'] = stats_blitz['ranking'].replace({'PLATINUM_PLUS':'all'}).str.lower()
stats_blitz['ranking'].unique()

array(['all', 'challenger', 'master', 'diamond', 'emerald', 'platinum',
       'gold', 'silver', 'bronze', 'iron'], dtype=object)

In [11]:
counters_opgg = pd.read_csv('datasets/counters2.csv')
counters_opgg

,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong,champion_name,tier,lane
0,1,"['garen', 'illaoi', 'drmundo', 'singed', 'zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['irelia', 'yasuo', 'yone', 'volibear', 'warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']",jax,all,top
1,2,"['warwick', 'kayle', 'malphite', 'monkeyking',...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","['ksante', 'akali', 'gwen', 'sion', 'sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']",fiora,all,top
2,4,"['singed', 'kayle', 'olaf', 'garen', 'drmundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['sylas', 'gragas', 'jayce', 'yasuo', 'akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']",poppy,all,top
3,2,"['vayne', 'kayle', 'urgot', 'drmundo', 'trundle']","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['gragas', 'yasuo', 'gwen', 'akali', 'zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']",singed,all,top
4,4,"['singed', 'vayne', 'volibear', 'mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['sylas', 'malphite', 'pantheon', 'tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']",zac,all,top
...,...,...,...,...,...,...,...,...,...,...
3651,NaN,"['blitzcrank', 'nautilus', 'seraphine', 'senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['yuumi', 'brand']","['71.05', '51.35']","['38', '37']",anivia,iron,support
3652,5,"['bard', 'maokai', 'veigar', 'velkoz', 'brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['milio', 'renata', 'pantheon', 'malphite', 's...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']",lulu,iron,support
3653,5,"['shaco', 'tahmkench', 'maokai', 'teemo', 'rak...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['hwei', 'janna']","['53.15', '52.7']","['286', '148']",yuumi,iron,support
3654,NaN,[],[],[],[],[],[],sivir,iron,support


In [12]:
mapping_champion_opgg = {
    'renata': 'Renata Glasc',
    'monkeyking': 'Wukong'
}

counters_opgg['champion_name'] = counters_opgg['champion_name'].replace(mapping_champion_opgg)
counters_opgg['champion_name'].unique()

array(['jax', 'fiora', 'poppy', 'singed', 'zac', 'ornn', 'cassiopeia',
       'illaoi', 'olaf', 'rengar', 'camille', 'akshan', 'vayne', 'nasus',
       'quinn', 'urgot', 'kled', 'garen', 'swain', 'kayle', 'malphite',
       'sylas', 'tahmkench', 'maokai', 'trundle', 'teemo', 'tryndamere',
       'warwick', 'riven', 'fiddlesticks', 'drmundo', 'neeko', 'udyr',
       'zed', 'shen', 'aatrox', 'mordekaiser', 'azir', 'aurelionsol',
       'jayce', 'varus', 'darius', 'chogath', 'ksante', 'lillia',
       'irelia', 'qiyana', 'yorick', 'anivia', 'kassadin', 'leblanc',
       'naafiri', 'vladimir', 'viktor', 'heimerdinger', 'galio',
       'renekton', 'karthus', 'kennen', 'gwen', 'annie', 'ryze', 'reksai',
       'sett', 'gragas', 'gnar', 'akali', 'rumble', 'volibear', 'rammus',
       'sion', 'malzahar', 'yasuo', 'yone', 'graves', 'kayn', 'pantheon',
       'lissandra', 'ahri', 'gangplank', 'kalista', 'ivern', 'shaco',
       'Wukong', 'karma', 'briar', 'shyvana', 'masteryi', 'veigar',
       

In [13]:
def normalize_string(s):
    # Rimuovi caratteri speciali, spazi e converti in minuscolo
    s = re.sub(r'[^a-zA-Z0-9]', '', s).lower()
    return s

In [14]:
def normalized_edit_distance(str1, str2):
    # Normalizza le stringhe prima di calcolare l'EOD
    normalized_str1 = normalize_string(str1)
    normalized_str2 = normalize_string(str2)

    # Calcola la differenza tra le due stringhe normalizzate
    d = difflib.Differ()
    diff = list(d.compare(normalized_str1, normalized_str2))

    # Calcola l'EOD contando le operazioni di inserimento, cancellazione e sostituzione
    edit_distance_count = sum(1 for op in diff if op.startswith('-') or op.startswith('+'))

    # Calcola la lunghezza massima tra le due stringhe normalizzate
    max_length = max(len(normalized_str1), len(normalized_str2))

    # Calcola l'EOD normalizzato
    normalized_eod = edit_distance_count / max_length

    return normalized_eod

In [15]:
def find_closest_string(target_str, string_list):
    if not string_list:
        return None

    # Inizializza con un valore alto per garantire un confronto corretto
    min_normalized_eod = float('inf')
    closest_string = None

    # Calcola l'EOD normalizzato per ogni stringa nella lista
    for candidate_str in string_list:
        normalized_eod = normalized_edit_distance(target_str, candidate_str)

        # Aggiorna il valore minimo e la stringa più vicina se necessario
        if normalized_eod < min_normalized_eod:
            min_normalized_eod = normalized_eod
            closest_string = candidate_str
    print('==================================================')
    print(f"Stringa di prima: {target_str}, Dopo: {closest_string}")
    print("normalized EOD: ", min_normalized_eod)
    return closest_string

In [16]:
#Prendo la lista dei campioni
champions_list = pd.read_csv('datasets/champions_list.csv')
champions_name_list = list(champions_list['name'])
champions_name_list

['Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'Aurelion Sol',
 'Azir',
 'Bard',
 "Bel'Veth",
 'Blitzcrank',
 'Brand',
 'Braum',
 'Briar',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 "Cho'Gath",
 'Corki',
 'Darius',
 'Diana',
 'Draven',
 'Dr. Mundo',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Hwei',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'Jarvan IV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 "Kai'Sa",
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 "Kha'Zix",
 'Kindred',
 'Kled',
 "Kog'Maw",
 "K'Sante",
 'LeBlanc',
 'Lee Sin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'Master Yi',
 'Milio',
 'Miss Fortune',
 'Wukong',
 'Mordekaiser',
 'Morgana',
 'Naafiri',
 'Nami',
 'Nasus',
 'Nautilus',
 'Nee

In [17]:
possible_champion_names = set(stats_lolalytics['champion_name']) | set(stats_blitz['champion_name']) | set(counters_opgg['champion_name'])
possible_champion_names

{'Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'Aurelion Sol',
 'AurelionSol',
 'Azir',
 'Bard',
 "Bel'Veth",
 'Belveth',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Briar',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 "Cho'Gath",
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'Dr. Mundo',
 'DrMundo',
 'Draven',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Hwei',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'Jarvan IV',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 "K'Sante",
 "Kai'Sa",
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 "Kha'Zix",
 'Khazix',
 'Kindred',
 'Kled',
 "Kog'Maw",
 'Kogmaw',
 'Ksante',
 'LeBlanc',
 'Lee Sin',
 'LeeSin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'Master Y

In [18]:
#Creo il dizionario mapping champion_name
mapping_champion_name = {}
champions_name_list_to_map = list(possible_champion_names)

for possible_name in possible_champion_names:
    mapping_champion_name[possible_name] = find_closest_string(possible_name,champions_name_list)

Stringa di prima: Milio, Dopo: Milio
normalized EOD:  0.0
Stringa di prima: rell, Dopo: Rell
normalized EOD:  0.0
Stringa di prima: Lulu, Dopo: Lulu
normalized EOD:  0.0
Stringa di prima: Akali, Dopo: Akali
normalized EOD:  0.0
Stringa di prima: jax, Dopo: Jax
normalized EOD:  0.0
Stringa di prima: chogath, Dopo: Cho'Gath
normalized EOD:  0.0
Stringa di prima: ziggs, Dopo: Ziggs
normalized EOD:  0.0
Stringa di prima: Zeri, Dopo: Zeri
normalized EOD:  0.0
Stringa di prima: udyr, Dopo: Udyr
normalized EOD:  0.0
Stringa di prima: vi, Dopo: Vi
normalized EOD:  0.0
Stringa di prima: Yasuo, Dopo: Yasuo
normalized EOD:  0.0
Stringa di prima: Qiyana, Dopo: Qiyana
normalized EOD:  0.0
Stringa di prima: Graves, Dopo: Graves
normalized EOD:  0.0
Stringa di prima: Master Yi, Dopo: Master Yi
normalized EOD:  0.0
Stringa di prima: drmundo, Dopo: Dr. Mundo
normalized EOD:  0.0
Stringa di prima: zoe, Dopo: Zoe
normalized EOD:  0.0
Stringa di prima: Shyvana, Dopo: Shyvana
normalized EOD:  0.0
Stringa d

Stringa di prima: lucian, Dopo: Lucian
normalized EOD:  0.0
Stringa di prima: Trundle, Dopo: Trundle
normalized EOD:  0.0
Stringa di prima: Gangplank, Dopo: Gangplank
normalized EOD:  0.0
Stringa di prima: Zoe, Dopo: Zoe
normalized EOD:  0.0
Stringa di prima: lulu, Dopo: Lulu
normalized EOD:  0.0
Stringa di prima: Cassiopeia, Dopo: Cassiopeia
normalized EOD:  0.0
Stringa di prima: Ornn, Dopo: Ornn
normalized EOD:  0.0
Stringa di prima: elise, Dopo: Elise
normalized EOD:  0.0
Stringa di prima: Karma, Dopo: Karma
normalized EOD:  0.0
Stringa di prima: lux, Dopo: Lux
normalized EOD:  0.0
Stringa di prima: Ekko, Dopo: Ekko
normalized EOD:  0.0
Stringa di prima: talon, Dopo: Talon
normalized EOD:  0.0
Stringa di prima: LeeSin, Dopo: Lee Sin
normalized EOD:  0.0
Stringa di prima: Vel'Koz, Dopo: Vel'Koz
normalized EOD:  0.0
Stringa di prima: Reksai, Dopo: Rek'Sai
normalized EOD:  0.0
Stringa di prima: yorick, Dopo: Yorick
normalized EOD:  0.0
Stringa di prima: Zilean, Dopo: Zilean
normalized 

In [19]:
#Rinomino
stats_lolalytics['champion_name'] = stats_lolalytics['champion_name'].replace(mapping_champion_name)
stats_blitz['champion_name'] = stats_blitz['champion_name'].replace(mapping_champion_name)
counters_opgg['champion_name'] = counters_opgg['champion_name'].replace(mapping_champion_name)

In [20]:
#Faccio il merge tra df e blitz
df_lolalytics_blitz = pd.merge(stats_lolalytics, stats_blitz, on = ['champion_name', 'ranking', 'lane'], how = 'left')
df_lolalytics_blitz

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,grade_y,counters_list,win_rate_list
0,1,Jax,S+,top,50.20,10.21,19.44,3,1737071,all,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']"
1,2,Fiora,S+,top,51.00,4.89,7.75,5,832573,all,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']"
2,3,Poppy,S,top,50.71,1.13,2.30,1,192796,all,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']"
3,4,Singed,S,top,52.80,1.53,0.48,4,260712,all,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']"
4,5,Zac,S,top,52.52,0.50,1.29,1,84558,all,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,D,support,36.99,0.43,0.79,-3,3393,iron,5,"['Heimerdinger', 'Renata Glasc', 'Ashe', 'Nami...","['14', '17', '20', '21', '22']"
3652,101,Lulu,D-,support,40.67,2.45,0.99,-9,19551,iron,4,"['Teemo', 'Renata Glasc', 'Zyra', ""Vel'Koz"", '...","['29', '31', '33', '33', '33']"
3653,102,Yuumi,D-,support,38.84,7.27,3.04,-40,57948,iron,4,"['Braum', 'Shaco', 'Swain', 'Bard', 'Alistar']","['29', '30', '30', '31', '32']"
3654,103,Sivir,D-,support,23.92,0.11,0.93,-2,899,iron,5,"['Morgana', 'Zyra', 'Hwei', 'Zilean', 'Sona']","['0', '0', '0', '0', '0']"


In [21]:
counters_opgg.rename(columns = {'tier': 'ranking'}, inplace=True)
counters_opgg

,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong,champion_name,ranking,lane
0,1,"['garen', 'illaoi', 'drmundo', 'singed', 'zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['irelia', 'yasuo', 'yone', 'volibear', 'warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']",Jax,all,top
1,2,"['warwick', 'kayle', 'malphite', 'monkeyking',...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","['ksante', 'akali', 'gwen', 'sion', 'sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']",Fiora,all,top
2,4,"['singed', 'kayle', 'olaf', 'garen', 'drmundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['sylas', 'gragas', 'jayce', 'yasuo', 'akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']",Poppy,all,top
3,2,"['vayne', 'kayle', 'urgot', 'drmundo', 'trundle']","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['gragas', 'yasuo', 'gwen', 'akali', 'zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']",Singed,all,top
4,4,"['singed', 'vayne', 'volibear', 'mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['sylas', 'malphite', 'pantheon', 'tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']",Zac,all,top
...,...,...,...,...,...,...,...,...,...,...
3651,NaN,"['blitzcrank', 'nautilus', 'seraphine', 'senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['yuumi', 'brand']","['71.05', '51.35']","['38', '37']",Anivia,iron,support
3652,5,"['bard', 'maokai', 'veigar', 'velkoz', 'brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['milio', 'renata', 'pantheon', 'malphite', 's...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']",Lulu,iron,support
3653,5,"['shaco', 'tahmkench', 'maokai', 'teemo', 'rak...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['hwei', 'janna']","['53.15', '52.7']","['286', '148']",Yuumi,iron,support
3654,NaN,[],[],[],[],[],[],Sivir,iron,support


In [22]:
lolalytics_blitz_counters = pd.merge(df_lolalytics_blitz, counters_opgg, on = ['champion_name', 'ranking', 'lane'], how = 'left')
lolalytics_blitz_counters

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,grade_y,counters_list,win_rate_list,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong
0,1,Jax,S+,top,50.20,10.21,19.44,3,1737071,all,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",1,"['garen', 'illaoi', 'drmundo', 'singed', 'zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['irelia', 'yasuo', 'yone', 'volibear', 'warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']"
1,2,Fiora,S+,top,51.00,4.89,7.75,5,832573,all,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",2,"['warwick', 'kayle', 'malphite', 'monkeyking',...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","['ksante', 'akali', 'gwen', 'sion', 'sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']"
2,3,Poppy,S,top,50.71,1.13,2.30,1,192796,all,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",4,"['singed', 'kayle', 'olaf', 'garen', 'drmundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['sylas', 'gragas', 'jayce', 'yasuo', 'akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']"
3,4,Singed,S,top,52.80,1.53,0.48,4,260712,all,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",2,"['vayne', 'kayle', 'urgot', 'drmundo', 'trundle']","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['gragas', 'yasuo', 'gwen', 'akali', 'zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']"
4,5,Zac,S,top,52.52,0.50,1.29,1,84558,all,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",4,"['singed', 'vayne', 'volibear', 'mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['sylas', 'malphite', 'pantheon', 'tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,D,support,36.99,0.43,0.79,-3,3393,iron,5,"['Heimerdinger', 'Renata Glasc', 'Ashe', 'Nami...","['14', '17', '20', '21', '22']",NaN,"['blitzcrank', 'nautilus', 'seraphine', 'senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['yuumi', 'brand']","['71.05', '51.35']","['38', '37']"
3652,101,Lulu,D-,support,40.67,2.45,0.99,-9,19551,iron,4,"['Teemo', 'Renata Glasc', 'Zyra', ""Vel'Koz"", '...","['29', '31', '33', '33', '33']",5,"['bard', 'maokai', 'veigar', 'velkoz', 'brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['milio', 'renata', 'pantheon', 'malphite', 's...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']"
3653,102,Yuumi,D-,support,38.84,7.27,3.04,-40,57948,iron,4,"['Braum', 'Shaco', 'Swain', 'Bard', 'Alistar']","['29', '30', '30', '31', '32']",5,"['shaco', 'tahmkench', 'maokai', 'teemo', 'rak...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['hwei', 'janna']","['53.15', '52.7']","['286', '148']"
3654,103,Sivir,D-,support,23.92,0.11,0.93,-2,899,iron,5,"['Morgana', 'Zyra', 'Hwei', 'Zilean', 'Sona']","['0', '0', '0', '0', '0']",NaN,[],[],[],[],[],[]


In [23]:

mapping_grade_lolalytics = {
    'S+': 0.666,
    'S': 1,
    'S-': 1.333,
    'A+': 1.666,
    'A': 2,
    'A-': 2.333,
    'B+': 2.666,
    'B': 3,
    'B-': 3.333,
    'C+': 3.666,
    'C': 4,
    'C-': 4.333,
    'D+': 4.666,
    'D': 5,
    'D-': 5.333

}

mapping_grade_opgg = {
    '0.0': 0.666
}

lolalytics_blitz_counters['grade_x'] = lolalytics_blitz_counters['grade_x'].replace(mapping_grade_lolalytics)
lolalytics_blitz_counters['grade'] = lolalytics_blitz_counters['grade'].apply(lambda x: 0.666 if x == 'op' else x)
lolalytics_blitz_counters['grade_y'] = lolalytics_blitz_counters['grade_y'].apply(lambda x: None if x == 10 else x)


lolalytics_blitz_counters

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,grade_y,counters_list,win_rate_list,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong
0,1,Jax,0.666,top,50.20,10.21,19.44,3,1737071,all,1.0,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",1,"['garen', 'illaoi', 'drmundo', 'singed', 'zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['irelia', 'yasuo', 'yone', 'volibear', 'warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']"
1,2,Fiora,0.666,top,51.00,4.89,7.75,5,832573,all,1.0,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",2,"['warwick', 'kayle', 'malphite', 'monkeyking',...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","['ksante', 'akali', 'gwen', 'sion', 'sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']"
2,3,Poppy,1.000,top,50.71,1.13,2.30,1,192796,all,2.0,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",4,"['singed', 'kayle', 'olaf', 'garen', 'drmundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['sylas', 'gragas', 'jayce', 'yasuo', 'akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']"
3,4,Singed,1.000,top,52.80,1.53,0.48,4,260712,all,2.0,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",2,"['vayne', 'kayle', 'urgot', 'drmundo', 'trundle']","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['gragas', 'yasuo', 'gwen', 'akali', 'zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']"
4,5,Zac,1.000,top,52.52,0.50,1.29,1,84558,all,3.0,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",4,"['singed', 'vayne', 'volibear', 'mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['sylas', 'malphite', 'pantheon', 'tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,5.000,support,36.99,0.43,0.79,-3,3393,iron,5.0,"['Heimerdinger', 'Renata Glasc', 'Ashe', 'Nami...","['14', '17', '20', '21', '22']",NaN,"['blitzcrank', 'nautilus', 'seraphine', 'senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['yuumi', 'brand']","['71.05', '51.35']","['38', '37']"
3652,101,Lulu,5.333,support,40.67,2.45,0.99,-9,19551,iron,4.0,"['Teemo', 'Renata Glasc', 'Zyra', ""Vel'Koz"", '...","['29', '31', '33', '33', '33']",5,"['bard', 'maokai', 'veigar', 'velkoz', 'brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['milio', 'renata', 'pantheon', 'malphite', 's...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']"
3653,102,Yuumi,5.333,support,38.84,7.27,3.04,-40,57948,iron,4.0,"['Braum', 'Shaco', 'Swain', 'Bard', 'Alistar']","['29', '30', '30', '31', '32']",5,"['shaco', 'tahmkench', 'maokai', 'teemo', 'rak...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['hwei', 'janna']","['53.15', '52.7']","['286', '148']"
3654,103,Sivir,5.333,support,23.92,0.11,0.93,-2,899,iron,5.0,"['Morgana', 'Zyra', 'Hwei', 'Zilean', 'Sona']","['0', '0', '0', '0', '0']",NaN,[],[],[],[],[],[]


In [24]:
lolalytics_blitz_counters['grade'] =lolalytics_blitz_counters['grade'].astype(float)
lolalytics_blitz_counters['grade'].unique()

array([1.   , 2.   , 4.   ,   nan, 3.   , 5.   , 0.666])

In [25]:
lolalytics_blitz_counters['grade_y'].unique()

array([ 1.,  2.,  3.,  4.,  5., nan])

In [26]:
def calculate_custom_mean(row):
    return row.mean(skipna=True)

In [27]:
lolalytics_blitz_counters['custom_grade'] = lolalytics_blitz_counters[['grade_x', 'grade_y', 'grade']].apply(calculate_custom_mean, axis = 1)
df_sorted_by_grade = lolalytics_blitz_counters.sort_values(by = 'custom_grade')

In [28]:
df_sorted_by_grade[(df_sorted_by_grade['custom_grade'] <= 1.5) & (df_sorted_by_grade['ranking'] == 'all')]

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,...,counters_list,win_rate_list,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong,custom_grade
0,1,Jax,0.666,top,50.20,10.21,19.44,3,1737071,all,...,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",1.0,"['garen', 'illaoi', 'drmundo', 'singed', 'zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['irelia', 'yasuo', 'yone', 'volibear', 'warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']",0.888667
95,2,Briar,0.666,jungle,52.76,7.03,30.62,28,1196654,all,...,"['Rammus', 'Trundle', ""Rek'Sai"", 'Nocturne']","['48', '50', '50', '50']",1.0,"['rammus', 'nocturne', 'trundle', 'jax']","['46.86', '49.03', '49.7', '49.95']","['36388', '77833', '17035', '19221']","['nidalee', 'sylas', 'nunu', 'rengar', 'zed']","['56.97', '56.96', '56.79', '56.65', '55.96']","['17893', '19113', '19076', '16376', '4548']",0.888667
168,2,Sylas,0.666,mid,50.46,9.82,19.95,6,1670966,all,...,"['Taliyah', 'Naafiri', 'Akshan', 'Galio', 'Mal...","['47', '48', '49', '49', '49']",1.0,"['heimerdinger', 'naafiri', 'malzahar', 'taliy...","['41.47', '45.18', '45.38', '45.63', '46.49']","['5353', '11710', '31083', '7639', '18181']","['malphite', 'hwei', 'zed', 'azir', 'ekko']","['57.78', '56.72', '53.21', '53.11', '52.96']","['7842', '80593', '64516', '28200', '26375']",0.888667
304,1,Rakan,0.666,support,51.41,7.03,3.44,10,1196555,all,...,"['Taric', 'Senna']","['50', '50']",1.0,"['zyra', 'senna', 'zilean', 'morgana', 'renata']","['48.36', '48.66', '48.7', '48.92', '48.98']","['35810', '68319', '12960', '46789', '22932']","['hwei', 'missfortune', 'yuumi', 'malphite', '...","['59.39', '57.34', '56.3', '55.49', '54.8']","['17895', '5173', '30363', '5212', '12640']",0.888667
97,4,Graves,1.000,jungle,50.60,10.26,8.73,7,1745278,all,...,"['Ivern', 'Fiddlesticks', 'Jax', ""Kha'Zix"", ""B...","['48', '49', '49', '49', '49']",1.0,"['rammus', 'fiddlesticks', 'ivern', 'briar', '...","['47.9', '48.18', '48.32', '48.51', '48.95']","['32421', '26292', '10542', '63143', '105833']","['monkeyking', 'sylas', 'zed', 'pantheon', 'sh...","['53.5', '53.4', '52.88', '52.65', '52.43']","['14181', '33990', '6297', '4997', '54862']",1.000000
264,2,Vayne,1.000,adc,51.55,10.63,9.80,18,1808716,all,...,['Seraphine'],['48'],1.0,"['seraphine', 'veigar', 'missfortune', 'swain']","['49.08', '49.11', '49.13', '49.24']","['7303', '7011', '129602', '5203']","['zeri', 'sivir', 'yasuo', 'xayah', 'kaisa']","['55.73', '54.17', '53.56', '53.25', '53.24']","['22259', '29980', '18659', '55637', '180644']",1.000000
266,4,Jhin,1.333,adc,51.34,15.90,3.73,22,2704350,all,...,['Vayne'],['50'],1.0,"['swain', 'seraphine', 'vayne', 'yasuo', 'twit...","['46.92', '48.1', '48.69', '49.11', '49.37']","['7700', '10230', '152466', '26277', '72550']","['zeri', 'xayah', 'aphelios', 'caitlyn', 'kaisa']","['53.36', '53.32', '52.91', '52.13', '51.78']","['30099', '78785', '61717', '340126', '233891']",1.111000
311,8,Xerath,1.333,support,52.36,6.80,16.60,19,1156446,all,...,"['Maokai', 'Janna', 'Milio', 'Sylas', 'Rakan']","['48', '49', '49', '50', '50']",1.0,"['maokai', 'taric', 'janna', 'sona', 'milio']","['47.86', '48.83', '49.42', '49.59', '49.67']","['10182', '4458', '21132', '13389', '19715']","['hwei', 'missfortune', 'veigar', 'ziggs', 'ma...","['61.83', '58.45', '57.66', '55.41', '54.69']","['18600', '8364', '4622', '4068', '6067']",1.111000
173,7,Fizz,1.333,mid,52.58,6.63,12.47,20,1127818,all,...,"['Lissandra', 'Kassadin', 'Sylas', 'Diana']","['47', '48', '50', '50']",1.0,"['lissandra', 'kassadin', 'galio', 'pantheon',...","['47.45', '47.7', '47.73', '49.16', '49.19']","['22701', '17563', '15915', '16978', '11474']","['hwei', 'aurelionsol', 'syndra', 'brand', 'zi...","['61.52', '56.22', '55.63', '55.36', '54.83']","['72157', '11792', '49279', '7535', '6551']",1.11100

In [29]:
def apply_function_to_list(input_string, my_function):
    # Converto la stringa in una lista di stringhe
    lista_originale = eval(input_string)
    
    # Applico la funzione a ciascun elemento della lista
    lista_modificata = [my_function(elemento) for elemento in lista_originale]
    
    # Converto la lista modificata in una stringa
    output_string = str(lista_modificata)
    
    return output_string

In [30]:
def get_correct_champion_name(x):
    if x == 'monkeyking':
        return 'Wukong'
    elif x == 'renata':
        return 'Renata Glasc'
    
    return mapping_champion_name[x] if x in list(mapping_champion_name.keys()) else x

In [31]:
mapping_champion_name.keys()

dict_keys(['Milio', 'rell', 'Lulu', 'Akali', 'jax', 'chogath', 'ziggs', 'Zeri', 'udyr', 'vi', 'Yasuo', 'Qiyana', 'Graves', 'Master Yi', 'drmundo', 'zoe', 'Shyvana', 'Yone', 'yone', 'Renekton', 'Senna', 'pantheon', 'Aatrox', 'Fizz', 'aurelionsol', 'ornn', 'Hwei', 'brand', 'aphelios', 'sylas', 'soraka', 'Gragas', 'Samira', 'briar', 'Ahri', 'urgot', 'lucian', 'Trundle', 'Gangplank', 'Zoe', 'lulu', 'Cassiopeia', 'Ornn', 'elise', 'Karma', 'lux', 'Ekko', 'talon', 'LeeSin', "Vel'Koz", 'Reksai', 'yorick', 'Zilean', 'Zyra', 'Lucian', 'Xin Zhao', 'Lee Sin', 'zilean', 'tryndamere', 'Orianna', 'malzahar', 'Swain', 'Ryze', 'azir', 'nidalee', 'evelynn', 'Pyke', 'Caitlyn', 'TahmKench', 'kalista', 'TwistedFate', 'Kled', 'Jayce', "Kai'Sa", 'galio', 'Twitch', 'hecarim', 'ivern', 'leesin', 'Malphite', 'Khazix', 'Taliyah', 'graves', 'khazix', 'alistar', 'Neeko', 'ezreal', 'Wukong', 'corki', 'Illaoi', 'Bard', 'velkoz', 'Morgana', 'zed', 'syndra', 'Kogmaw', 'nunu', 'viktor', 'Nami', 'taliyah', 'Alistar', 's

In [32]:
#Devo rinominare i nomi dei counters e dei strong
lolalytics_blitz_counters['counter'] =lolalytics_blitz_counters['counter'].apply(lambda x: apply_function_to_list(x, get_correct_champion_name))
lolalytics_blitz_counters['counter']

0       ['Garen', 'Illaoi', 'Dr. Mundo', 'Singed', 'Zac']
1       ['Warwick', 'Kayle', 'Malphite', 'Wukong', 'Il...
2       ['Singed', 'Kayle', 'Olaf', 'Garen', 'Dr. Mundo']
3       ['Vayne', 'Kayle', 'Urgot', 'Dr. Mundo', 'Trun...
4       ['Singed', 'Vayne', 'Volibear', 'Mordekaiser',...
                              ...                        
3651    ['Blitzcrank', 'Nautilus', 'Seraphine', 'Senna...
3652     ['Bard', 'Maokai', 'Veigar', "Vel'Koz", 'Brand']
3653    ['Shaco', 'Tahm Kench', 'Maokai', 'Teemo', 'Ra...
3654                                                   []
3655    ['Heimerdinger', 'Leona', 'Blitzcrank', 'Rakan...
Name: counter, Length: 3656, dtype: object

In [33]:
lolalytics_blitz_counters['strong'] =lolalytics_blitz_counters['strong'].apply(lambda x: apply_function_to_list(x, get_correct_champion_name))
lolalytics_blitz_counters['strong']

0       ['Irelia', 'Yasuo', 'Yone', 'Volibear', 'Warwi...
1           ["K'Sante", 'Akali', 'Gwen', 'Sion', 'Sylas']
2          ['Sylas', 'Gragas', 'Jayce', 'Yasuo', 'Akali']
3             ['Gragas', 'Yasuo', 'Gwen', 'Akali', 'Zac']
4       ['Sylas', 'Malphite', 'Pantheon', 'Tryndamere'...
                              ...                        
3651                                   ['Yuumi', 'Brand']
3652    ['Milio', 'Renata Glasc', 'Pantheon', 'Malphit...
3653                                    ['Hwei', 'Janna']
3654                                                   []
3655         ['Karma', 'Teemo', 'Nami', 'Lulu', 'Maokai']
Name: strong, Length: 3656, dtype: object

In [34]:
lolalytics_blitz_counters

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,...,counters_list,win_rate_list,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong,custom_grade
0,1,Jax,0.666,top,50.20,10.21,19.44,3,1737071,all,...,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",1.0,"['Garen', 'Illaoi', 'Dr. Mundo', 'Singed', 'Zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['Irelia', 'Yasuo', 'Yone', 'Volibear', 'Warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']",0.888667
1,2,Fiora,0.666,top,51.00,4.89,7.75,5,832573,all,...,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",2.0,"['Warwick', 'Kayle', 'Malphite', 'Wukong', 'Il...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","[""K'Sante"", 'Akali', 'Gwen', 'Sion', 'Sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']",1.222000
2,3,Poppy,1.000,top,50.71,1.13,2.30,1,192796,all,...,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",4.0,"['Singed', 'Kayle', 'Olaf', 'Garen', 'Dr. Mundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['Sylas', 'Gragas', 'Jayce', 'Yasuo', 'Akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']",2.333333
3,4,Singed,1.000,top,52.80,1.53,0.48,4,260712,all,...,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",2.0,"['Vayne', 'Kayle', 'Urgot', 'Dr. Mundo', 'Trun...","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['Gragas', 'Yasuo', 'Gwen', 'Akali', 'Zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']",1.666667
4,5,Zac,1.000,top,52.52,0.50,1.29,1,84558,all,...,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",4.0,"['Singed', 'Vayne', 'Volibear', 'Mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['Sylas', 'Malphite', 'Pantheon', 'Tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']",2.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,5.000,support,36.99,0.43,0.79,-3,3393,iron,...,"['Heimerdinger', 'Renata Glasc', 'Ashe', 'Nami...","['14', '17', '20', '21', '22']",NaN,"['Blitzcrank', 'Nautilus', 'Seraphine', 'Senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['Yuumi', 'Brand']","['71.05', '51.35']","['38', '37']",5.000000
3652,101,Lulu,5.333,support,40.67,2.45,0.99,-9,19551,iron,...,"['Teemo', 'Renata Glasc', 'Zyra', ""Vel'Koz"", '...","['29', '31', '33', '33', '33']",5.0,"['Bard', 'Maokai', 'Veigar', ""Vel'Koz"", 'Brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['Milio', 'Renata Glasc', 'Pantheon', 'Malphit...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']",4.777667
3653,102,Yuumi,5.333,support,38.84,7.27,3.04,-40,57948,iron,...,"['Braum', 'Shaco', 'Swain', 'Bard', 'Alistar']","['29', '30', '30', '31', '32']",5.0,"['Shaco', 'Tahm Kench', 'Maokai', 'Teemo', 'Ra...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['Hwei', 'Janna']","['53.15', '52.7']","['286', '148']",4.777667
3654,103,Sivir,5.333,support,23.92,0.11,0.93,-2,899,iron,...,"['Morgana', 'Zyra', 'Hwei', 'Zilean', 'Sona']","['0', '0', '0', '0', '0']",NaN,[],[],[],[],[],[],5.166500


In [35]:
# I decide to drop counters list from blitz
lolalytics_blitz_counters.drop(['counters_list', 'win_rate_list'], axis = 1, inplace=True)
lolalytics_blitz_counters

,position,champion_name,grade_x,lane,win_rate,pick_rate,ban_rate,pbi,games_played,ranking,grade_y,grade,counter,win_rate_weak,games_played_weak,strong,win_rate_strong,games_played_strong,custom_grade
0,1,Jax,0.666,top,50.20,10.21,19.44,3,1737071,all,1.0,1.0,"['Garen', 'Illaoi', 'Dr. Mundo', 'Singed', 'Zac']","['44.01', '44.53', '44.85', '44.86', '45.8']","['100862', '50803', '28349', '14508', '5380']","['Irelia', 'Yasuo', 'Yone', 'Volibear', 'Warwi...","['55.01', '54.88', '54.02', '53.73', '52.95']","['33167', '19581', '89243', '33202', '9495']",0.888667
1,2,Fiora,0.666,top,51.00,4.89,7.75,5,832573,all,1.0,2.0,"['Warwick', 'Kayle', 'Malphite', 'Wukong', 'Il...","['46.23', '46.78', '47.01', '47.43', '47.72']","['7564', '12532', '27335', '2686', '20090']","[""K'Sante"", 'Akali', 'Gwen', 'Sion', 'Sylas']","['55.97', '54.75', '54.5', '54.46', '54.18']","['17031', '12587', '14770', '16710', '3721']",1.222000
2,3,Poppy,1.000,top,50.71,1.13,2.30,1,192796,all,2.0,4.0,"['Singed', 'Kayle', 'Olaf', 'Garen', 'Dr. Mundo']","['45.86', '46.13', '46.16', '46.34', '46.54']","['1378', '3061', '2684', '6752', '1981']","['Sylas', 'Gragas', 'Jayce', 'Yasuo', 'Akali']","['57.19', '55.65', '55.36', '54.8', '54.49']","['932', '859', '4801', '2208', '2841']",2.333333
3,4,Singed,1.000,top,52.80,1.53,0.48,4,260712,all,2.0,2.0,"['Vayne', 'Kayle', 'Urgot', 'Dr. Mundo', 'Trun...","['46.07', '46.59', '48.82', '49.12', '49.55']","['1973', '5666', '3380', '4127', '12864']","['Gragas', 'Yasuo', 'Gwen', 'Akali', 'Zac']","['59.3', '58.36', '57.44', '57.33', '57.23']","['1135', '2971', '4641', '3035', '692']",1.666667
4,5,Zac,1.000,top,52.52,0.50,1.29,1,84558,all,3.0,4.0,"['Singed', 'Vayne', 'Volibear', 'Mordekaiser',...","['42.77', '47.64', '47.9', '48.39', '48.92']","['692', '594', '1885', '2145', '646']","['Sylas', 'Malphite', 'Pantheon', 'Tryndamere'...","['58.61', '57.48', '57.23', '57.13', '56.78']","['633', '3304', '858', '1248', '1032']",2.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,5.000,support,36.99,0.43,0.79,-3,3393,iron,5.0,NaN,"['Blitzcrank', 'Nautilus', 'Seraphine', 'Senna...","['28.21', '33.33', '36.36', '36.76', '41.77']","['39', '51', '55', '68', '79']","['Yuumi', 'Brand']","['71.05', '51.35']","['38', '37']",5.000000
3652,101,Lulu,5.333,support,40.67,2.45,0.99,-9,19551,iron,4.0,5.0,"['Bard', 'Maokai', 'Veigar', ""Vel'Koz"", 'Brand']","['33.33', '34', '36.36', '36.73', '36.84']","['36', '50', '33', '49', '171']","['Milio', 'Renata Glasc', 'Pantheon', 'Malphit...","['57.14', '53.85', '53.03', '52.38', '52.1']","['35', '39', '66', '63', '119']",4.777667
3653,102,Yuumi,5.333,support,38.84,7.27,3.04,-40,57948,iron,4.0,5.0,"['Shaco', 'Tahm Kench', 'Maokai', 'Teemo', 'Ra...","['31.82', '33.33', '33.83', '35.94', '37.7']","['88', '90', '133', '217', '427']","['Hwei', 'Janna']","['53.15', '52.7']","['286', '148']",4.777667
3654,103,Sivir,5.333,support,23.92,0.11,0.93,-2,899,iron,5.0,NaN,[],[],[],[],[],[],5.166500


In [70]:
lolalytics_blitz_counters.to_csv('datasets/complete.csv', index = False)